### And welcome to Week 4, Day 3 - more LangGraph..

In [ ]:
# Importing necessary libraries

import requests
import os
import gradio as gr
from typing import TypedDict
from dotenv import load_dotenv
from IPython.display import Image, display
from typing import Annotated

# Using Amazon Bedrock Models
from langchain_aws import ChatBedrockConverse 
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph
from langgraph.prebuilt import ToolNode, tools_condition

In [ ]:
# Loading up environment variables
# Using amazon bedrock, values like aws secret key, access key, bedrock api key and aws region name should exist
load_dotenv(override=True)

Set up LangSmith via the link below to enable tracing:

https://langsmith.com

### Next, here is a useful function in LangChain community:

In [ ]:
from langchain_community.utilities import GoogleSerperAPIWrapper

serper = GoogleSerperAPIWrapper()
serper.run("What is the capital of Kenya?")

#### LangChain wrapper class for converting functions into Tools

In [ ]:
from langchain.agents import Tool

search_tool = Tool(
    name="Internet_Search_Tool",
    func=serper.run,
    description="Used to get information from the internet."
)

#### Invoking the tool in the langchain way

In [ ]:
search_tool.invoke("What is the capital of Kenya?")

#### Push Notification Tool

This is a push notification tool, that when invoked it sends an sms push notification to the phone.

In [ ]:
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, 
                    data = {"token": pushover_token, 
                            "user": pushover_user, 
                            "message": text})

In [ ]:
push_notification_tool = Tool(
    name="Send_Push_Notification",
    func=push,
    description="Used to send a push notification to the phone."
)

#push_notification_tool.invoke("Hello. This is a test notication")

Using TypedDict instead of BaseModel for the State object

When we implement tools, we always need to make 2 changes to the code:

1. Changes to provide the tools to Amazon Bedrock in json when we make the call

2. Changes to handle the results back: look for the model staying that the finish_reason=="tool_calls" and then retrieve the call, run the function, provide the results.

#### Bring the tools together

In [ ]:
tools = [search_tool, push_notification_tool]

In [ ]:
# Step 1: Define the State object
class State(TypedDict):
    messages: Annotated[list, add_messages]

In [ ]:
# Step 2: Start the Graph Builder with this State class
graph_builder = StateGraph(State)

In [ ]:
# Invoking the chat bedrock api using amazon nova micro model

llm = ChatBedrockConverse(model_id="amazon.nova-micro-v1:0")

# Binding the LLMs with tools
llm_with_tools = llm.bind_tools(tools)

In [ ]:
# Step 3: Create a Node
from langgraph import graph

def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

In [ ]:
# Step 4: Create Edges

graph_builder.add_conditional_edges("chatbot", tools_condition, "tools")

# Any time a tool is called, we return to the chatbot to decide the next step
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")


In [ ]:
# Step 5: Compile the Graph and display the flow
graph = graph_builder.compile()
display(Image(graph.get_graph().draw_mermaid_png()))

#### Creating the chatbot function and displating gradio intereface

In [ ]:
def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]})
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()

### Adding Memory

We have this whole Graph maintaining the state and appending to the state.

Why isn't this handling memory?

This is a crucial point for understanding LangGraph

> A super-step can be considered a single iteration over the graph nodes. Nodes that run in parallel are part of the same super-step, while nodes that run sequentially belong to separate super-steps.


One "Super-Step" of the graph represents one invocation of passing messages between agents.

In idomatic LangGraph, you call invoke to run your graph for each super-step; for each interaction.

The `reducer` handles state updates automatically within one super-step, but not between them.

That is what `checkpointing` achieves.

In [ ]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatBedrockConverse(model_id="amazon.nova-micro-v1:0")
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
# Pick a thread to use as the key for this conversation
config = {"configurable": {"thread_id": "1"}}

# Chatbot config
def chat(user_input: str, history):
    results = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return results["messages"][-1].content

# # launching gradio
gr.ChatInterface(chat, type="messages").launch()

In [ ]:
# getting the state of the config
graph.get_state(config)

In [ ]:
# Listing the state of the config, Most recent first

list(graph.get_state_history(config))

#### LangGraph gives you tools to set the state back to a prior point in time, to branch off:

```
config = {"configurable": {"thread_id": "1", "checkpoint_id": ...}}
graph.invoke(None, config=config)
```

And this allows you to build stable systems that can be recovered and rerun from any prior checkpoint.

 #### Memory - SQL
 And now let's store in SQL

 And this is the power of LangGraph.

In [ ]:
# importing and setting up sqlite db

import sqlite3
from langgraph.checkpoint.sqlite import SqliteSaver

db_path = "memory.db"
connection = sqlite3.connect(db_path, check_same_thread=False)
sql_memory = SqliteSaver(connection)

In [ ]:
# Steps 1 and 2
graph_builder = StateGraph(State)

# Step 3
llm = ChatBedrockConverse(model_id="amazon.nova-micro-v1:0")
tools = [search_tool, push_notification_tool]
llm_with_tools = llm.bind_tools(tools)

def chatbot(state: State):
    print(state)
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

graph_builder.add_node("chatbot", chatbot)
graph_builder.add_node("tools", ToolNode(tools=tools))

# Step 4
graph_builder.add_conditional_edges( "chatbot", tools_condition, "tools")
graph_builder.add_edge("tools", "chatbot")
graph_builder.add_edge(START, "chatbot")

# Step 5
graph = graph_builder.compile(checkpointer=sql_memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
config = {"configurable": {"thread_id": "2"}}

def chat(user_input: str, history):
    result = graph.invoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content


gr.ChatInterface(chat, type="messages").launch()